In [1]:
from expression_atlas_db import base, load_db, settings

In [2]:
Session = base.configure(settings.db_connection_string)
session = Session()

In [5]:
[*session.query(base.Study.alembic_id).distinct()]

[('1feece4d9e7f',), ('55614015ac51',), ('87e23735af2d',)]

In [7]:
[*session.query(base.Study.velia_id).filter(base.Study.public == False)]

[('SRP108496',),
 ('SRP173298',),
 ('SRP226476',),
 ('SRP223858',),
 ('SRP303290',),
 ('SRP021193',),
 ('SRP304149',),
 ('SRP321796',),
 ('SRP379573',),
 ('GSE112087',),
 ('GSE120178',),
 ('GSE122459',),
 ('GSE139358',),
 ('GSE83687',),
 ('SRP042228',),
 ('SRP055438',),
 ('ERP106487',),
 ('SRP458076',),
 ('SRP102685',),
 ('SRP221484',),
 ('SRP155976',),
 ('SRP414798',)]

In [8]:
# Run update, pulling qc from s3 staging and updating qc dependent on qc file.

load_db.update_studies_qc(
    connection_string=settings.db_connection_string,
)

INFO:root:Updating QC sheet.
INFO:aiobotocore.credentials:Found credentials from IAM Role: AWSScopedAccess
INFO:root:Reading QC sheet: velia-piperuns-dev/expression_atlas/staging/qc.4.txt.
INFO:root:Updated studies with QC sheet: velia-piperuns-dev/expression_atlas/staging/qc.4.txt.


In [9]:
[*session.query(base.Study.velia_id).filter(base.Study.public == False)]

[('SRP173298',),
 ('ERP106487',),
 ('GSE112087',),
 ('GSE122459',),
 ('GSE139358',),
 ('GSE83687',),
 ('SRP042228',),
 ('SRP055438',),
 ('SRP102685',),
 ('SRP221484',),
 ('SRP226476',),
 ('SRP303290',),
 ('GSE120178',),
 ('SRP304149',),
 ('SRP321796',),
 ('SRP155976',),
 ('SRP379573',),
 ('SRP414798',)]